## 가위바위보_구별기
-----

### 반복 사용되는 모듈 임포트 

In [9]:
from PIL import Image
import matplotlib.pyplot as plt
import os, glob, cv2

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


### 가위, 바위, 보 이미지 전처리

In [52]:
def resize_images(img_path): 
    for folder in os.listdir(img_path):
        if folder not in ['rock', 'scissor', 'paper']: 
            continue   
        images=glob.glob(img_path + f"/{folder}/*.jpg")  
        print(f"{len(images)} {folder} images to be resized.")
        # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
        target_size=(28,28)
        for img in images:
            old_img=cv2.imread(img,cv2.IMREAD_GRAYSCALE)
            new_img=cv2.resize(old_img,target_size,Image.ANTIALIAS)
            cv2.imwrite(img,new_img)
        print(f"{len(images)} {folder} images resized.")

In [55]:
folder_name = input("test/train 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
resize_images(image_dir_path)

print(f"{folder_name} 이미지 resize 완료!")

test/train 중 하나를 입력하세요train
1108 paper images to be resized.
1108 paper images resized.
1105 rock images to be resized.
1105 rock images resized.
994 scissor images to be resized.
994 scissor images resized.
train 이미지 resize 완료!


### 전처리된 이미지 행렬화

In [56]:
import numpy as np

def load_data(img_path, number_of_data=300):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=1
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size, color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        if idx > (number_of_data//3)-1:
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
        
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        if idx > (number_of_data//3*2)-1:
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        if idx > number_of_data-1:
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [75]:
folder_name = input("train/test 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
print(folder_name)
if folder_name == "train":
    (x_train, y_train)=load_data(image_dir_path,3300)
    x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
    print(f"x_train shape: {x_train.shape}")
    print(f"y_train shape: {y_train.shape}")
else: 
    (x_test, y_test)=load_data(image_dir_path)
    x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
    x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
    x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)
    print(f"x_test reshaped: {x_test_reshaped.shape}")
    print(f"y_test reshaped: {x_test_reshaped.shape}")

train/test 중 하나를 입력하세요train
train
학습데이터(x_train)의 이미지 개수는 3207 입니다.
x_train shape: (3300, 28, 28, 1)
y_train shape: (3300,)


In [59]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                51264     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

### 모델 학습

In [60]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
110/110 [==============================] - 5s 19ms/step - loss: 1.8645 - accuracy: 0.9378
Epoch 2/10
110/110 [==============================] - 0s 3ms/step - loss: 0.0017 - accuracy: 0.9998
Epoch 3/10
110/110 [==============================] - 0s 3ms/step - loss: 1.6834e-04 - accuracy: 1.0000
Epoch 4/10
110/110 [==============================] - 0s 3ms/step - loss: 4.3182e-06 - accuracy: 1.0000
Epoch 5/10
110/110 [==============================] - 0s 3ms/step - loss: 1.1098e-06 - accuracy: 1.0000
Epoch 6/10
110/110 [==============================] - 0s 3ms/step - loss: 6.4206e-07 - accuracy: 1.0000
Epoch 7/10
110/110 [==============================] - 0s 3ms/step - loss: 6.3927e-07 - accuracy: 1.0000
Epoch 8/10
110/110 [==============================] - 0s 3ms/step - loss: 5.7818e-07 - accuracy: 1.0000
Epoch 9/10
110/110 [==============================] - 0s 3ms/step - loss: 7.8375e-07 - accuracy: 1.0000
Epoch 10/10
110/110 [==============================] - 0s 3ms/step - lo

In [83]:
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print(f'test_loss: {test_loss}')
print(f'test_accuracy: {test_accuracy}')

10/10 - 0s - loss: 2.2033 - accuracy: 0.3333
test_loss: 2.2032649517059326
test_accuracy: 0.3333333432674408


In [58]:
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))

Before Reshape - x_train_norm shape: (3500, 28, 28, 1)


NameError: name 'x_test_norm' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()